In [28]:
import sys
import re 
import numpy as np 
import pandas as pd
import music21
from glob import glob
import IPython
from tqdm import tqdm
import pickle
import tensorflow as tf
import play
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Bidirectional, BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from music21 import converter, instrument, note, chord, stream
from keras.layers import Activation, Dense, LSTM, Dropout, Flatten

In [29]:
musicas = glob('Rap/*.mid')
musicas = musicas[::]

In [31]:
def obter_notas():
    print("Obtendo notas")
    notas = []
    for arquivo in musicas:
        # convertendo o arquivo .mid para um objeto stream
        midi = converter.parse(arquivo)
        notas_para_analisar = []
        
        try:
            # Dada uma única stream, particiona em uma parte para cada instrumento único
            partes = instrument.partitionByInstrument(midi)
        except:
            pass
        if partes:  # se houver partes de instrumento
            notas_para_analisar = partes.parts[0].recurse()
        else:
            notas_para_analisar = midi.flat.notes
    
        for elemento in notas_para_analisar: 
            if isinstance(elemento, note.Note):
                # se o elemento for uma nota, extraia o tom
                notas.append(str(elemento.pitch))
            elif(isinstance(elemento, chord.Chord)):
                # se o elemento for um acorde, adicione a forma normal do acorde 
                # (uma lista de inteiros) à lista de notas
                notas.append('.'.join(str(n) for n in elemento.normalOrder))
    with open('dados/notas', 'wb') as caminho_arquivo:
        pickle.dump(notas, caminho_arquivo)
    return notas


In [32]:
def preparar_sequencias(notas, n_vocab): 
    comprimento_sequencia = 100

    nomes_tons = sorted(set(item for item in notas))
    tom_para_inteiro = dict((tom, numero) for numero, tom in enumerate(nomes_tons))

    entrada_rede = []
    saida_rede = []

    for i in range(0, len(notas) - comprimento_sequencia, 1):
        sequencia_entrada = notas[i: i + comprimento_sequencia]
        sequencia_saida = notas[i + comprimento_sequencia]
        entrada_rede.append([tom_para_inteiro[char] for char in sequencia_entrada])
        saida_rede.append(tom_para_inteiro[sequencia_saida])
    
    numero_padroes = len(entrada_rede)
    entrada_rede = np.reshape(entrada_rede, (numero_padroes, comprimento_sequencia, 1))
    entrada_rede = entrada_rede / float(n_vocab)
    saida_rede = tf.keras.utils.to_categorical(saida_rede)
    
    return (entrada_rede, saida_rede)


In [33]:
def criar_rede_neural(entrada_rede, n_vocab): 
    modelo = Sequential()
    modelo.add(LSTM(128, input_shape=entrada_rede.shape[1:], return_sequences=True))
    modelo.add(Dropout(0.2))
    modelo.add(LSTM(128, return_sequences=True))
    modelo.add(Flatten())
    modelo.add(Dense(256))
    modelo.add(Dense(256))
    modelo.add(Dropout(0.3))
    modelo.add(Dense(n_vocab))
    modelo.add(Activation('softmax'))
    modelo.compile(loss='categorical_crossentropy', optimizer='adam')

    return modelo


In [34]:
import matplotlib.pyplot as plt

def treinar(modelo, entrada_rede, saida_rede, epocas): 

    caminho_arquivo = './modelo_teste.h5'
    ponto_verificacao = ModelCheckpoint(caminho_arquivo, monitor='loss', verbose=0, save_best_only=True)
    
    historico = modelo.fit(entrada_rede, saida_rede, epochs=epocas, batch_size=32, callbacks=[ponto_verificacao])

    # Plotando a perda (Loss)
    plt.plot(historico.history['loss'])
    plt.title('Perda no Treinamento')
    plt.xlabel('Época')
    plt.ylabel('Perda')
    plt.show()


In [35]:
def treinar_rede_neural():
    """
    Obter notas
    Gerar sequências de entrada e saída
    Criar um modelo
    Treinar o modelo para as épocas fornecidas
    """
    epocas = 100
    
    notas = obter_notas()
    print('Notas processadas')

    n_vocab = len(set(notas))
    print('Vocabulário gerado')

    entrada_rede, saida_rede = preparar_sequencias(notas, n_vocab)
    print('Entrada e Saída processadas')

    modelo = criar_rede_neural(entrada_rede, n_vocab)
    print('Modelo criado')
    print('Treinamento em andamento')
    
    treinar(modelo, entrada_rede, saida_rede, epocas)
    print('Treinamento concluído')
    return modelo


In [36]:
### Treinar o modelo
modelo = treinar_rede_neural()
modelo.save_weights('modelo_treinado.h5')


Obtendo notas
Notas processadas
Vocabulário gerado
Entrada e Saída processadas
Modelo criado
Treinamento em andamento
Epoch 1/100
323/323 [==============================] - 37s 106ms/step - loss: 4.5667
Epoch 2/100
323/323 [==============================] - 34s 105ms/step - loss: 4.2279
Epoch 3/100
323/323 [==============================] - 34s 106ms/step - loss: 3.9636
Epoch 4/100
323/323 [==============================] - 35s 107ms/step - loss: 3.7489
Epoch 5/100
323/323 [==============================] - 36s 111ms/step - loss: 3.4996
Epoch 6/100
323/323 [==============================] - 35s 108ms/step - loss: 3.2074
Epoch 7/100
323/323 [==============================] - 35s 109ms/step - loss: 2.9536
Epoch 8/100
323/323 [==============================] - 34s 106ms/step - loss: 2.7382
Epoch 9/100
323/323 [==============================] - 35s 109ms/step - loss: 2.5409
Epoch 10/100
323/323 [==============================] - 35s 109ms/step - loss: 2.3148
Epoch 11/100
323/323 [=========

In [ ]:
def obter_sequencias_entrada(notas, nomes_tons, n_vocab):
    
    # mapear entre notas e números e vice-versa
    nota_para_inteiro = dict((nota, numero) for numero, nota in enumerate(nomes_tons))

    comprimento_sequencia = 100
    entrada_rede = []
    for i in range(0, len(notas) - comprimento_sequencia, 1):
        sequencia_entrada = notas[i:i + comprimento_sequencia]
        entrada_rede.append([nota_para_inteiro[char] for char in sequencia_entrada])
    
    entrada_rede = np.reshape(entrada_rede, (len(entrada_rede), comprimento_sequencia, 1))
    
    return entrada_rede


In [ ]:
def gerar_notas(modelo, entrada_rede, nomes_tons, n_vocab, temperatura=2.0):
    
    # Escolher um número inteiro aleatório
    inicio = np.random.randint(0, len(entrada_rede)-1)

    inteiro_para_nota = dict((numero, nota) for numero, nota in enumerate(nomes_tons))
    
    # Escolher uma sequência aleatória da entrada como ponto de partida para a previsão
    padrao = list(entrada_rede[inicio])
    resultado_predicao = []
    
    print('Gerando notas........')

    # Gerar 300 notas
    for indice_nota in range(300):
        entrada_predicao = np.reshape(padrao, (1, len(padrao), 1))
        padrao = np.array(padrao)
        entrada_predicao = entrada_predicao / float(n_vocab)

        # Ajuste da temperatura
        predicao = modelo.predict(entrada_predicao, verbose=0)[0]
        predicao = np.log(predicao) / temperatura
        exp_preds = np.exp(predicao)
        predicao = exp_preds / np.sum(exp_preds)

        # Escolha aleatória com base nas probabilidades ajustadas
        indice = np.random.choice(len(predicao), p=predicao)
        resultado = inteiro_para_nota[indice]

        # Armazenar a saída prevista
        resultado_predicao.append(resultado)

        padrao = np.append(padrao, indice)
        padrao = padrao[1:]

    print('Notas geradas...')
    print(resultado_predicao)
    return resultado_predicao


In [ ]:
def criar_midi(resultado_predicao):
    
    offset = 0
    notas_saida = []

    # criar objetos de nota e acorde com base nos valores gerados pelo modelo
    for padrao in resultado_predicao:
        if ('.' in padrao) or padrao.isdigit():
            notas_no_acorde = padrao.split('.')
            notas = []
            for nota_atual in notas_no_acorde:
                nova_nota = note.Note(int(nota_atual))
                nova_nota.storedInstrument = instrument.Piano()
                notas.append(nova_nota)
            novo_acorde = chord.Chord(notas)
            novo_acorde.offset = offset
            notas_saida.append(novo_acorde)
        # padrao é uma nota
        else:
            nova_nota = note.Note(padrao)
            nova_nota.offset = offset
            nova_nota.storedInstrument = instrument.Piano()
            notas_saida.append(nova_nota)

        # aumentar o offset a cada iteração para que as notas não se sobreponham
        offset += 0.5

    fluxo_midi = stream.Stream(notas_saida)
    
    print('Salvando arquivo de saída como midi....')

    fluxo_midi.write('midi', fp='saida_teste5.mid')


In [ ]:
def generate():
    """ Generate a piano midi file """
    #load the notes used to train the model
    with open('data/notes', 'rb') as filepath:
        notes = pickle.load(filepath)

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # Get all pitch names
    n_vocab = len(set(notes))
    
    print('Initiating music generation process.......')
    
    network_input = get_inputSequences(notes, pitchnames, n_vocab)
    normalized_input = network_input / float(n_vocab)
    model = create_network(normalized_input, n_vocab)
    print('Loading Model weights.....')
    model.load_weights('./modelo_teste.h5')
    print('Model Loaded')
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output)

In [ ]:

generate()

Initiating music generation process.......
Loading Model weights.....
Model Loaded
Generating notes........


C:\Users\PC\AppData\Local\Temp\ipykernel_16588\1127633650.py:22: RuntimeWarning: divide by zero encountered in log
  prediction = np.log(prediction) / temperature


Notes Generated...
['4.6.11', '2.6.9', '2.6.9', '2.5', '5.10', '2.5', '5.10', '10.2.5', '7.10', '2.7', '7.10', '2.7', 'G4', '4.6', 'A3', '4.6', 'A3', '4.6', 'A3', '2.6', 'A3', 'D4', 'A3', '2.6', 'A3', '4.6', 'A3', '4.6', 'A3', '4.6', 'A3', '2.6', 'A3', '2.6', 'A3', '2.6', 'A3', '2.6.9', '5.9.0', '2.5.9', 'E-5', '2.4', 'A3', '11.2.6', 'A3', '9.1.4', '5.10', '7.9', 'A3', '2.6.9', 'A3', 'F5', 'B-3', '5.7', 'D2', 'G3', '6.9', '2.5', 'C#5', 'D2', 'C5', '6.10.1', '11.2.6', '2.5.9', 'E-6', 'A4', 'F#4', '11.2.6', '2.6.9', 'B-3', 'F#4', '5.9.0', '2.5.9', 'G4', 'G5', 'E-4', 'G4', 'G5', 'G3', 'A4', 'C6', '10.1', 'G4', '10.0', 'A4', '10.0', 'E-4', '0.2.6.8', '0.2.5', '7.10.0', 'G3', '8.10.2', '7.10.0', '7.10.0', 'A4', '1.7', '5', 'C5', 'C3', 'F#4', 'F#4', 'F#4', 'G4', '9', 'F3', '10.1', 'F3', 'E4', 'F3', 'F3', 'F4', 'C3', 'F3', 'F3', 'F3', 'A4', 'F3', 'A4', 'F3', 'F3', '10.2.5', 'F3', 'F4', 'G4', 'F4', 'B-4', 'D4', 'F3', '5', 'G4', 'E-4', 'G4', 'F4', 'F4', 'F3', 'F3', '6.9.0', '10.2.5', 'D6', 'B4'

In [ ]:
play.play_midi('test_output4.mid')

Music file test_output4.mid loaded!
